### Lab2 : Working with Spark SQL

#### We will review :

1. Loading CSV file formats using SparkSession
2. Creating DataFrame without inferring Schema 
3. Creating DataFrame inferring Schema 
4. Doing some preliminary analysis using Spark SQL on this dataset
5. Creating UDFs (User Defined Functions) and using them on the dataset
5. Saving a DataFrame into partitioned parquet files format

#### Small (Lab) Dataset :

* Air flight data - subset of ~ 100 MB (for demonstration purposes)
* Available in the IE cluster @: /data/shared/spark/flight_data/csv_tiny

#### Larger Dataset (Further Labs) :

* Air flight data - subset of ~ 2.5 GB (for cluster operation purposes)
* Available in the IE cluster @: /data/shared/spark/flight_data/csv_small


In [ ]:
# First Let's start by :
# 1. Definining SPARK_HOME variable 
# 2. Using findspark to  let us work with Spark installation in the cluster

In [ ]:
import os
print(os.environ['SPARK_HOME'])

In [ ]:
import findspark
findspark.init()
import pyspark

In [ ]:
# Create a SparkSession and specify configuration
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[1]") \
    .appName("Spark-SQL-Lab2") \
    .getOrCreate()

In [ ]:
dataset_path="/data/shared/spark/flight_data/csv_tiny/"

In [ ]:
# Read in all available data files into a data frame
df = spark.read \
    .csv("file://"+dataset_path+"*.csv")   

### Now check the data schema

In [ ]:
df.printSchema()

* Ok , but the column names are not very telling. 
* How to improve this? , by telling Spark to use the header ( if exists )

In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv("file://"+dataset_path+"*.csv")

In [ ]:
df.printSchema()

* Better , but still one caveat though , all values are interpreted as string, while some of them (actually most), are of numeric nature ( e.g ) Year , Month , Flight Number
* How to improve this ?, by either telling Spark what schema to use OR telling it to infer the Schema of the data
* Note : Asking Spark to infer schema may have a performance impact depending on the number of rows required to infer the schema

In [ ]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("file://"+dataset_path+"*.csv")

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
# Register a table named flights for later SQL queries
df.registerTempTable("flights")

#### Select the following columns from the full dataset

    Year
    Month
    DayOfMonth
    DayOfWeek
    FlightNum
    Origin
    Carrier
    Dest ( destination )
    DepTime ( departure time )
    DepDelayMinutes ( departure delay )
    ArrTime ( arrival time )
    ArrDelay ( arrival delay )
    Cancelled
    CancellationCode
    AirTime
    Distance


In [ ]:
# Register the table for later SQL queries
df.registerTempTable("flights")

#### Worth Noting

* registerTempTable() creates an in-memory table avaialble within cluster in which it was created. The data is stored using Hive's in-memory columnar format and will only 'live' for the duration of the session.

* saveAsTable() creates a permanent, physical table stored using the Parquet format. This table is accessible to all clusters including external clusters and in between sessions. The table metadata including the location of the file(s) is stored within the Hive metastore.

In [ ]:
df_subset=spark.sql(
    "select " 
    +"year,month,dayofmonth,dayofweek,"
    +"flightnum,origin,carrier,dest,deptime,depdelay,"
    +"arrtime,arrdelay,cancelled,cancellationcode,"
    +"airtime,distance "
    +"FROM flights"
    )
# OR 
# selection=["year,month,dayofmonth,dayofweek,"
#       "flightnum,origin,dest,deptime,depdelay,"
#       "arrtime,arrdelay,cancelled,cancellationcode,"
#       "airtime,distance "]
# info.select(selection)

In [ ]:
# Cache this DataFrame
df_subset.cache()

In [ ]:
# Show the first 5 rows of the subset data to get a feeling of what to expect
df_subset.head(5)

In [ ]:
df_subset.take(3)

### Do some SQL queries ( use both the DataFrame and direct SQL queries )

1. Find the number of departing flights from a given airport
2. Find the total number of delayed flights on a given airport
3. Find the average delay per airport
4. Find the top 5 airports with the highest average delays
5. Find the worst airport in terms of total nb cancelled flights (cancelled=1.0) 

In [ ]:
# how many records do we have in total?
total=df_subset.count()
print('Total nb.of flights: %d' % total)
# OR in SQL
spark.sql("select COUNT(*) from flights").show()

In [ ]:

# 1.2. how many flights and delayed
def statsByAirport(airport_id,df):
    from_id=df.filter(df['origin']==airport_id)
    delayed=from_id.filter(df['depdelay']>=15.0)
    ndep=from_id.count()
    ndel=delayed.count()
    return (ndep,ndel)
    
airport='JFK'

n,m=statsByAirport(airport,df_subset)

print('Departing from %s : %d ' %(airport,n))
print('Delayed   from %s : %d ' %(airport,m))
print('Delayed Percentage : %f %%' %((m/n)*100))

In [ ]:
# 3. Average delay per flight on an airport
def averageDelay(airport_id,df):
    from_id=df.filter(df['origin']==airport_id)
    return from_id.select('depdelay').describe() # returns a dataframe with descriptive statistics

airport='JFK'
df=averageDelay(airport,df_subset)

print('Airport : %s ' %(airport))
print('Average delay : %f min' %(float(df.collect()[1]['depdelay'])))

In [ ]:
# 4. Top 5 airports with highest average delay : actually easier here with SQL AVG
query = "SELECT origin,AVG(depdelay) FROM flights GROUP BY origin ORDER BY avg(depdelay) DESC"
df_delays=spark.sql(query)
df_delays.show()

In [ ]:
# 5. The worst airport in terms of cancelled flights
# Create a function that simply sums the total number of flights cancelled on a given airport
# ---> Remember you should 'weight' cancelled against total , in order not to bias the result
#
def cancellations(airport_id):
    # USING DF: 
    #     idf=df_subset.filter(df_subset['origin']==airport_id)
    #     return idf.filter(idf['cancelled']==1.0).count()
    # USING SQL:
    query="select * from flights where origin=='"+airport_id+"'" +" and cancelled==1.0"
    return spark.sql(query).select('cancelled').count()

In [ ]:
# Register the function with Spark SQL as User Defined Function
spark.udf.register("cancellations", lambda x : cancellations(x))
query = "SELECT origin,cancellations(origin) AS score FROM flights GROUP BY origin ORDER BY score DESC"
spark.sql(query).show()

* Save this dataframe in parquet (columnar) format for boost in loading performance
* In order to do we want to 'be clever' and partition the data by specific atributes , in this case
* Year and Month

In [ ]:
# Save the data into my HOME
# IMPORTANT NOTE: we are partinioning (structuring)
# by relevant factors in our data , in this case year and month
# can be used to naturally save this data.
my_home=os.environ['HOME']
out_dir="airline_data"
df_subset.write.partitionBy(
        "Year","Month"
    ).parquet(
        "file://"
        + my_home
        +'/'
        + out_dir,
        mode='overwrite'
    )
print('Done!')

In [ ]:
# Read CSV data into a dictionary of DataFrame : try to infer schema directly from the data
import itertools
year_list = ['2014']
month_list = ['1','2','3','4','5','6','7','8','9','10','11','12']

dict_df = {}

for (year_str,month_str) in list(itertools.product(year_list,month_list)):
    year_month_str = '%s_%s'%(year_str,month_str)
    print('Reading input data for year:%s month:%s'%(year_str,month_str))
    df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("file://"+dataset_path+"On_Time_On_Time_Performance_%s.csv"%(year_month_str))  
    df.cache()
    dict_df[year_month_str]=df
print('Done!')